In [1]:
import requests
import boto3
import json
import os
import sys
import argparse
import importlib
import transformers
import torch
import pathlib
import awswrangler as wr
from IPython.display import display
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import FrameworkProcessor
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.session import get_execution_role


# Adding ../01_modules or ./01_modules to the system path so that we can load modules from 
# there as well
if '__file__' in globals():
    script_dir = pathlib.Path(__file__).parent.resolve()
else:
    script_dir = pathlib.Path().absolute()
modules_path_in_dev = os.path.abspath(os.path.join(script_dir, '..', '01_modules'))
modules_path_in_prod = os.path.abspath(os.path.join(script_dir, '01_modules'))
if os.path.exists(modules_path_in_dev):
    sys.path.append(modules_path_in_dev)
if os.path.exists(modules_path_in_prod):
    sys.path.append(modules_path_in_prod)


# # Jupyter only reads a local module the first time after 
# # kernel start. Re-running a cell with 
# # "from mymodulename import *" would not change
# # anything, even if the imported module has since changed.
# # As a workaround, we need to directly load the module, 
# # use importlib.reload to reload it and then import * 
import utils
_ = importlib.reload(utils)
import config
_ = importlib.reload(config)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
config.py loaded: v0.1
utils.py loaded: v0.2.12
utils.py loaded: v0.2.12
config.py loaded: v0.1


# Problem statement

My initial hypothesis was that the SemanticScholar S2Orc dataset and the 
OpenAlex Works dataset can be easily connected. This is partially true. S2Orc 
does have a MAG (Microsoft Academic Graph) ID reference. MAG is discontinued, 
but OpenAlex was created as a successor to MAG, and took over all the MAG IDs 
and just prefixed them with a "W" (as in Works). So given that the 
SemanticScholar S2Orc dataset has a good coverage for the MAG id, and the 
equivalent of it is the primary id in the OpenAlex Works dataset, I assumed, 
they would match nicely, so I could combine the full text from SemanticScholar 
with the domain/field/subfield/topic categorization of OpenAlex.

But looking az the first join attempt, it looks like there are very few matches 
percentage-wise. The matches that we have are correct, but a lot of records 
with filled in ID values don't match. 

Now I have a lot of questions:
 - How come that these datasets don't match each other?
 - How many S2 records have MAG id without having a match in OA?
 - How many OA records don't have a match in SA?
 - There is an OA dataset called merged_ids, could that be of any help?
 - Is there any other shared ID that could be used?
 - Is there any other way to match millions of records, like title similarity, etc?

# S2 stats

### Number of S2 records with MAG id

In [10]:
utils.pd_set_options(cols=500)
wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT * FROM "02_stg".base_semanticscholar_s2orcv2
),
flagged AS (
    SELECT
        *,
        CASE WHEN COALESCE(id_mag, '')='' THEN 0 ELSE 1 END AS has_id_mag
    FROM
        base_semanticscholar_s2orcv2_
),
grouped AS (
    SELECT
        has_id_mag,
        COUNT(*) / 1000000.0 AS c_mill,
        COUNT(*) * 100.0 / (SELECT COUNT(*) FROM flagged) AS p,
        (SELECT COUNT(*) FROM flagged) / 1000000.0 as total_mill
    FROM
        flagged
    GROUP BY
        has_id_mag
    ORDER BY
        has_id_mag
)
SELECT * FROM grouped
 """, '02_stg')

,has_id_mag,c_mill,p,total_mill
0,0,5.073262,43.698149,11.609787
1,1,6.536525,56.301851,11.609787


So we know that 6.53M S2 records (56.3% of the total 11.6M records) has id_mag set.

### Number of S2 records with DOI id
This might be a possible fallback connection method, so let's see the same as above.

In [11]:
utils.pd_set_options(cols=500)
wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT * FROM "02_stg".base_semanticscholar_s2orcv2
),
flagged AS (
    SELECT
        *,
        CASE WHEN COALESCE(id_doi, '')='' THEN 0 ELSE 1 END AS has_id_doi
    FROM
        base_semanticscholar_s2orcv2_
),
grouped AS (
    SELECT
        has_id_doi,
        COUNT(*) / 1000000.0 AS c_mill,
        COUNT(*) * 100.0 / (SELECT COUNT(*) FROM flagged) AS p,
        (SELECT COUNT(*) FROM flagged) / 1000000.0 as total_mill
    FROM
        flagged
    GROUP BY
        has_id_doi
    ORDER BY
        has_id_doi
)
SELECT * FROM grouped
 """, '02_stg')

,has_id_doi,c_mill,p,total_mill
0,0,0.661094,5.694282,11.609787
1,1,10.948693,94.305718,11.609787


Not surprisingly, this has a much higher coverage 10.95M S2 records (94.3% of the total 11.6M records) has id_doi set.

# OA stats

In [12]:
utils.pd_set_options(cols=500)
wr.athena.read_sql_query("""
WITH
base_openalex_works_reduced_ AS (
    SELECT * FROM "02_stg"."base_openalex_works_reduced"
),
flagged AS (
    SELECT
        *,
        CASE WHEN COALESCE(id_doi, '')='' THEN 0 ELSE 1 END AS has_id_doi
    FROM
        base_openalex_works_reduced_
),
grouped AS (
    SELECT
        has_id_doi,
        COUNT(*) / 1000000.0 AS c_mill,
        COUNT(*) * 100.0 / (SELECT COUNT(*) FROM flagged) AS p,
        (SELECT COUNT(*) FROM flagged) / 1000000.0 as total_mill
    FROM
        flagged
    GROUP BY
        has_id_doi
    ORDER BY
        has_id_doi
)
SELECT * FROM grouped
 """, '02_stg')

,has_id_doi,c_mill,p,total_mill
0,0,93.581745,34.653243,270.051911
1,1,176.470166,65.346757,270.051911


First of all, we can see that the OA dataset has more than 20 times more records.  
Out of this 270M records, 176.47M (65.3%) has DOI id set. 

# S2-OA join stats

### Joining on S2.MAG_ID to OA.OA_ID

In [20]:
# Confirming that the join works as expected:
utils.pd_set_options(cols=500)
wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT * FROM "02_stg".base_semanticscholar_s2orcv2
),
base_openalex_works_reduced_ AS (
SELECT * FROM "02_stg"."base_openalex_works_reduced"
),
flagged_semanticscholar_s2orcv2 AS (
    SELECT
        *,
        CASE WHEN COALESCE(id_mag, '')='' THEN 0 ELSE 1 END AS has_id_mag,
        CASE WHEN COALESCE(id_doi, '')='' THEN 0 ELSE 1 END AS has_id_doi
    FROM
        base_semanticscholar_s2orcv2_
),
s2oa_join_on_mag_openalex_id AS (
    SELECT
        flagged_semanticscholar_s2orcv2.title AS semanticscholar_title, 
        flagged_semanticscholar_s2orcv2.id_semanticscholar AS semanticscholar_id_semanticscholar,
        flagged_semanticscholar_s2orcv2.id_mag AS semanticscholar_id_mag,
        flagged_semanticscholar_s2orcv2.id_doi AS semanticscholar_id_doi,                         -- Note: This is case-sensitive
        flagged_semanticscholar_s2orcv2.has_id_mag,
        flagged_semanticscholar_s2orcv2.has_id_doi,
        base_openalex_works_reduced_.title AS openalex_title,
        base_openalex_works_reduced_.id_openalex AS openalex_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_id_doi,                                 -- Note: This is forced lower-case
        CASE WHEN base_openalex_works_reduced_.id_openalex IS NULL THEN 0 ELSE 1 END AS join_worked
    FROM
        flagged_semanticscholar_s2orcv2
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        flagged_semanticscholar_s2orcv2.id_mag = base_openalex_works_reduced_.id_openalex
)
SELECT * FROM s2oa_join_on_mag_openalex_id WHERE join_worked=1 LIMIT 3
 """, '02_stg')

,semanticscholar_title,semanticscholar_id_semanticscholar,semanticscholar_id_mag,semanticscholar_id_doi,has_id_mag,has_id_doi,openalex_title,openalex_id_openalex,openalex_id_doi,join_worked
0,Are we all exploiters?*,225303433,3082394426,10.1111/phpr.12730,1,1,Are we all exploiters?*,3082394426,10.1111/phpr.12730,1
1,Spatial Fano resonance of a dielectric microsphere impinged on by a Bessel beam,220845731,3045856572,10.1364/JOSAB.424927,1,1,Spatial Fano resonance of a dielectric microsphere impinged on by a Bessel beam,3045856572,10.1364/josab.424927,1
2,Can Explicit Written Corrective Feedback Develop Grammatical and Lexical Accuracy of Saudi EFL Learners?,149864623,2906712829,10.7575/AIAC.IJELS.V.6N.4P.16,1,1,Can Explicit Written Corrective Feedback Develop Grammatical and Lexical Accuracy of Saudi EFL Learners?,2906712829,10.7575/aiac.ijels.v.6n.4p.16,1


In [21]:
utils.pd_set_options(cols=500)
wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT * FROM "02_stg".base_semanticscholar_s2orcv2
),
base_openalex_works_reduced_ AS (
SELECT * FROM "02_stg"."base_openalex_works_reduced"
),
flagged_semanticscholar_s2orcv2 AS (
    SELECT
        *,
        CASE WHEN COALESCE(id_mag, '')='' THEN 0 ELSE 1 END AS has_id_mag,
        CASE WHEN COALESCE(id_doi, '')='' THEN 0 ELSE 1 END AS has_id_doi
    FROM
        base_semanticscholar_s2orcv2_
),
s2oa_join_on_mag_openalex_id AS (
    SELECT
        flagged_semanticscholar_s2orcv2.title AS semanticscholar_title, 
        flagged_semanticscholar_s2orcv2.id_semanticscholar AS semanticscholar_id_semanticscholar,
        flagged_semanticscholar_s2orcv2.id_mag AS semanticscholar_id_mag,
        flagged_semanticscholar_s2orcv2.id_doi AS semanticscholar_id_doi,                         -- Note: This is case-sensitive
        flagged_semanticscholar_s2orcv2.has_id_mag AS semanticscholar_has_id_mag,
        flagged_semanticscholar_s2orcv2.has_id_doi AS semanticscholar_has_id_doi,
        base_openalex_works_reduced_.title AS openalex_title,
        base_openalex_works_reduced_.id_openalex AS openalex_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_id_doi,                                 -- Note: This is forced lower-case
        CASE WHEN base_openalex_works_reduced_.id_openalex IS NULL THEN 0 ELSE 1 END AS join_worked
    FROM
        flagged_semanticscholar_s2orcv2
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        flagged_semanticscholar_s2orcv2.id_mag = base_openalex_works_reduced_.id_openalex
),
grouped AS (
    SELECT
        semanticscholar_has_id_mag,
        join_worked,
        COUNT(*) / 1000000.0 AS c_mill,
        COUNT(*) * 100.0 / (SELECT COUNT(*) FROM s2oa_join_on_mag_openalex_id) AS p,
        (SELECT COUNT(*) FROM s2oa_join_on_mag_openalex_id) / 1000000.0 as total_mill
    FROM
        s2oa_join_on_mag_openalex_id
    GROUP BY
        semanticscholar_has_id_mag,
        join_worked
    ORDER BY
        semanticscholar_has_id_mag,
        join_worked
)
SELECT * FROM grouped
 """, '02_stg')

,semanticscholar_has_id_mag,join_worked,c_mill,p,total_mill
0,0,0,5.073262,43.698149,11.609787
1,1,0,0.413841,3.564587,11.609787
2,1,1,6.122684,52.737264,11.609787


Interesting. It seems that the prevous ad-hoc join was faulty, since this time the majority of the joins worked.  

However, this is not wasted energy. We still have almost half the records that we could not join using the MAG ID, so it might worth exploring if we can join those using DOI or the OA merged_ids.

### Joining on S2.DOI_ID to OA.DOI_ID

In [2]:
# Confirming that the join works as expected:
utils.pd_set_options(cols=500)
wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT * FROM "02_stg".base_semanticscholar_s2orcv2
),
base_openalex_works_reduced_ AS (
SELECT * FROM "02_stg"."base_openalex_works_reduced"
),
flagged_semanticscholar_s2orcv2 AS (
    SELECT
        *,
        CASE WHEN COALESCE(id_mag, '')='' THEN 0 ELSE 1 END AS has_id_mag,
        CASE WHEN COALESCE(id_doi, '')='' THEN 0 ELSE 1 END AS has_id_doi
    FROM
        base_semanticscholar_s2orcv2_
),
s2oa_join_on_doi_id AS (
    SELECT
        flagged_semanticscholar_s2orcv2.title AS semanticscholar_title, 
        flagged_semanticscholar_s2orcv2.id_semanticscholar AS semanticscholar_id_semanticscholar,
        flagged_semanticscholar_s2orcv2.id_mag AS semanticscholar_id_mag,
        flagged_semanticscholar_s2orcv2.id_doi AS semanticscholar_id_doi,                         -- Note: This is case-sensitive
        flagged_semanticscholar_s2orcv2.has_id_mag,
        flagged_semanticscholar_s2orcv2.has_id_doi,
        base_openalex_works_reduced_.title AS openalex_title,
        base_openalex_works_reduced_.id_openalex AS openalex_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_id_doi,                                 -- Note: This is forced lower-case
        CASE WHEN base_openalex_works_reduced_.id_openalex IS NULL THEN 0 ELSE 1 END AS join_worked
    FROM
        flagged_semanticscholar_s2orcv2
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        flagged_semanticscholar_s2orcv2.id_doi = base_openalex_works_reduced_.id_doi
)
SELECT * FROM s2oa_join_on_doi_id WHERE join_worked=1 LIMIT 3
 """, '02_stg')

,semanticscholar_title,semanticscholar_id_semanticscholar,semanticscholar_id_mag,semanticscholar_id_doi,has_id_mag,has_id_doi,openalex_title,openalex_id_openalex,openalex_id_doi,join_worked
0,A sulfurization method for creating the buffer-layers current flow diverter architecture in REBa2Cu3O7 coated conductors,264062317,<NA>,10.1088/1361-6668/ad01ec,0,1,A sulfurization method for creating the buffer-layers current flow diverter architecture in REBa<sub>2</sub>Cu<sub>3</sub>O<sub>7</sub> coated conductors,4387483590,10.1088/1361-6668/ad01ec,1
1,The Power of Students: Using Positioning Theory and Frame Analysis to Explore Power Dynamics in Mentoring Relationships,278994850,<NA>,10.5334/pme.1662,0,1,The Power of Students: Using Positioning Theory and Frame Analysis to Explore Power Dynamics in Mentoring Relationships,4410822614,10.5334/pme.1662,1
2,A Pest to Mental Health? Exploring the Link between Exposure to Agrichemicals in Farmers and Mental Health,129943947,2938595944,10.3390/ijerph16081327,1,1,A Pest to Mental Health? Exploring the Link between Exposure to Agrichemicals in Farmers and Mental Health,2938595944,10.3390/ijerph16081327,1


This is a problem with sampling results: I will only get confirmation on something being found, but not on the join finding everything it should. This time, I already found out on accident that the S2 DOI is Case Sensitive, while the OA DOI is lowercase, so I know a lot of ids would not match, but lets see the numbers.

In [4]:
utils.pd_set_options(cols=500)
wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT * FROM "02_stg".base_semanticscholar_s2orcv2
),
base_openalex_works_reduced_ AS (
SELECT * FROM "02_stg"."base_openalex_works_reduced"
),
flagged_semanticscholar_s2orcv2 AS (
    SELECT
        *,
        CASE WHEN COALESCE(id_mag, '')='' THEN 0 ELSE 1 END AS has_id_mag,
        CASE WHEN COALESCE(id_doi, '')='' THEN 0 ELSE 1 END AS has_id_doi
    FROM
        base_semanticscholar_s2orcv2_
),
s2oa_join_on_doi_id AS (
    SELECT
        flagged_semanticscholar_s2orcv2.title AS semanticscholar_title, 
        flagged_semanticscholar_s2orcv2.id_semanticscholar AS semanticscholar_id_semanticscholar,
        flagged_semanticscholar_s2orcv2.id_mag AS semanticscholar_id_mag,
        flagged_semanticscholar_s2orcv2.id_doi AS semanticscholar_id_doi,                         -- Note: This is case-sensitive
        flagged_semanticscholar_s2orcv2.has_id_mag AS semanticscholar_has_id_mag,
        flagged_semanticscholar_s2orcv2.has_id_doi AS semanticscholar_has_id_doi,
        base_openalex_works_reduced_.title AS openalex_title,
        base_openalex_works_reduced_.id_openalex AS openalex_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_id_doi,                                 -- Note: This is forced lower-case
        CASE WHEN base_openalex_works_reduced_.id_openalex IS NULL THEN 0 ELSE 1 END AS join_worked
    FROM
        flagged_semanticscholar_s2orcv2
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        flagged_semanticscholar_s2orcv2.id_doi = base_openalex_works_reduced_.id_doi
),
grouped AS (
    SELECT
        semanticscholar_has_id_doi,
        join_worked,
        COUNT(*) / 1000000.0 AS c_mill,
        COUNT(*) * 100.0 / (SELECT COUNT(*) FROM s2oa_join_on_doi_id) AS p,
        (SELECT COUNT(*) FROM s2oa_join_on_doi_id) / 1000000.0 as total_mill
    FROM
        s2oa_join_on_doi_id
    GROUP BY
        semanticscholar_has_id_doi,
        join_worked
    ORDER BY
        semanticscholar_has_id_doi,
        join_worked
)
SELECT * FROM grouped
 """, '02_stg')

,semanticscholar_has_id_doi,join_worked,c_mill,p,total_mill
0,0,0,0.661094,5.691963,11.614516
1,1,0,2.757111,23.738492,11.614516
2,1,1,8.196311,70.569544,11.614516


(Note: I most probably will go and modify the base models to contain lowercase DOIs, so re-runnig the above will not provide the same problematic numbers in the future.)  
Regardless, we already have a better match ratio than we had with MAGs. 8.2M S2 record (70.6% of all the 11.6M records) could be matched.  
But let's see it with the lowercase issue fixed.

In [5]:
utils.pd_set_options(cols=500)
wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT *, LOWER(id_doi) AS id_doi_lower FROM "02_stg".base_semanticscholar_s2orcv2
),
base_openalex_works_reduced_ AS (
SELECT *, LOWER(id_doi) AS id_doi_lower FROM "02_stg"."base_openalex_works_reduced"
),
flagged_semanticscholar_s2orcv2 AS (
    SELECT
        *,
        CASE WHEN COALESCE(id_mag, '')='' THEN 0 ELSE 1 END AS has_id_mag,
        CASE WHEN COALESCE(id_doi, '')='' THEN 0 ELSE 1 END AS has_id_doi
    FROM
        base_semanticscholar_s2orcv2_
),
s2oa_join_on_doi_id AS (
    SELECT
        flagged_semanticscholar_s2orcv2.title AS semanticscholar_title, 
        flagged_semanticscholar_s2orcv2.id_semanticscholar AS semanticscholar_id_semanticscholar,
        flagged_semanticscholar_s2orcv2.id_mag AS semanticscholar_id_mag,
        flagged_semanticscholar_s2orcv2.id_doi AS semanticscholar_id_doi,                         -- Note: This is case-sensitive
        flagged_semanticscholar_s2orcv2.id_doi_lower AS semanticscholar_id_doi_lower,             -- Note: lowered
        flagged_semanticscholar_s2orcv2.has_id_mag AS semanticscholar_has_id_mag,
        flagged_semanticscholar_s2orcv2.has_id_doi AS semanticscholar_has_id_doi,
        base_openalex_works_reduced_.title AS openalex_title,
        base_openalex_works_reduced_.id_openalex AS openalex_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_id_doi,                                 -- Note: This is forced lower-case
        base_openalex_works_reduced_.id_doi_lower AS openalex_id_doi_lower,                     -- Note: lowered (just to be sure)
        CASE WHEN base_openalex_works_reduced_.id_doi IS NULL THEN 0 ELSE 1 END AS join_worked
    FROM
        flagged_semanticscholar_s2orcv2
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        flagged_semanticscholar_s2orcv2.id_doi_lower = base_openalex_works_reduced_.id_doi_lower
),
grouped AS (
    SELECT
        semanticscholar_has_id_doi,
        join_worked,
        COUNT(*) / 1000000.0 AS c_mill,
        COUNT(*) * 100.0 / (SELECT COUNT(*) FROM s2oa_join_on_doi_id) AS p,
        (SELECT COUNT(*) FROM s2oa_join_on_doi_id) / 1000000.0 as total_mill
    FROM
        s2oa_join_on_doi_id
    GROUP BY
        semanticscholar_has_id_doi,
        join_worked
    ORDER BY
        semanticscholar_has_id_doi,
        join_worked
)
SELECT * FROM grouped
 """, '02_stg')

,semanticscholar_has_id_doi,join_worked,c_mill,p,total_mill
0,0,0,0.661094,5.690325,11.617861
1,1,0,0.072717,0.625907,11.617861
2,1,1,10.884050,93.683768,11.617861


This is amazing, this is already 93.7% coverage.  
But let's see if combining the 2 joins could even improve this.

In [13]:
utils.pd_set_options(cols=500)
wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT *, LOWER(id_doi) AS id_doi_lower FROM "02_stg".base_semanticscholar_s2orcv2
),
base_openalex_works_reduced_ AS (
SELECT *, LOWER(id_doi) AS id_doi_lower FROM "02_stg"."base_openalex_works_reduced"
),
flagged_semanticscholar_s2orcv2 AS (
    SELECT
        *,
        CASE WHEN COALESCE(id_mag, '')='' THEN 0 ELSE 1 END AS has_id_mag,
        CASE WHEN COALESCE(id_doi, '')='' THEN 0 ELSE 1 END AS has_id_doi,
        CASE WHEN COALESCE(id_doi, '')='' AND COALESCE(id_mag, '')='' THEN 0 ELSE 1 END AS has_id_mag_or_doi
    FROM
        base_semanticscholar_s2orcv2_
),
s2oa_join_on_doi_id AS (
    SELECT
        flagged_semanticscholar_s2orcv2.title AS semanticscholar_title, 
        flagged_semanticscholar_s2orcv2.id_semanticscholar AS semanticscholar_id_semanticscholar,
        flagged_semanticscholar_s2orcv2.id_mag AS semanticscholar_id_mag,
        flagged_semanticscholar_s2orcv2.id_doi AS semanticscholar_id_doi,                         -- Note: This is case-sensitive
        flagged_semanticscholar_s2orcv2.id_doi_lower AS semanticscholar_id_doi_lower,             -- Note: lowered
        flagged_semanticscholar_s2orcv2.has_id_mag AS semanticscholar_has_id_mag,
        flagged_semanticscholar_s2orcv2.has_id_doi AS semanticscholar_has_id_doi,
        flagged_semanticscholar_s2orcv2.has_id_mag_or_doi AS semanticscholar_has_id_mag_or_doi,
        base_openalex_works_reduced_.title AS openalex_doi_title,
        base_openalex_works_reduced_.id_openalex AS openalex_doi_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_doi_id_doi,                                 -- Note: This is forced lower-case
        base_openalex_works_reduced_.id_doi_lower AS openalex_doi_id_doi_lower,                     -- Note: lowered (just to be sure)
        CASE WHEN base_openalex_works_reduced_.id_doi IS NULL THEN 0 ELSE 1 END AS join_worked_on_doi
    FROM
        flagged_semanticscholar_s2orcv2
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        flagged_semanticscholar_s2orcv2.id_doi_lower = base_openalex_works_reduced_.id_doi_lower
),
s2oa_join_on_mag_openalex_id AS (
    SELECT
        s2oa_join_on_doi_id.semanticscholar_title, 
        s2oa_join_on_doi_id.semanticscholar_id_semanticscholar,
        s2oa_join_on_doi_id.semanticscholar_id_mag,
        s2oa_join_on_doi_id.semanticscholar_id_doi,                         -- Note: This is case-sensitive
        s2oa_join_on_doi_id.semanticscholar_id_doi_lower,                   -- Note: lowered
        s2oa_join_on_doi_id.semanticscholar_has_id_mag,
        s2oa_join_on_doi_id.semanticscholar_has_id_doi,
        s2oa_join_on_doi_id.semanticscholar_has_id_mag_or_doi,

        s2oa_join_on_doi_id.openalex_doi_title,
        s2oa_join_on_doi_id.openalex_doi_id_openalex,
        s2oa_join_on_doi_id.openalex_doi_id_doi,                           -- Note: This is forced lower-case
        s2oa_join_on_doi_id.openalex_doi_id_doi_lower,                     -- Note: lowered (just to be sure)
        s2oa_join_on_doi_id.join_worked_on_doi,
        
        base_openalex_works_reduced_.title AS openalex_mag_title,
        base_openalex_works_reduced_.id_openalex AS openalex_mag_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_mag_id_doi,                                 -- Note: This is forced lower-case
        base_openalex_works_reduced_.id_doi_lower AS openalex_mag_id_doi_lower,                     -- Note: lowered (just to be sure)
        CASE WHEN base_openalex_works_reduced_.id_openalex IS NULL THEN 0 ELSE 1 END AS join_worked_on_mag
    FROM
        s2oa_join_on_doi_id
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        s2oa_join_on_doi_id.join_worked_on_doi=0 AND -- only joining those through MAG that we couldn't through DOI
        s2oa_join_on_doi_id.semanticscholar_id_mag = base_openalex_works_reduced_.id_openalex
),
grouped AS (
    SELECT
        semanticscholar_has_id_mag_or_doi,
        semanticscholar_has_id_doi,
        join_worked_on_doi,
        semanticscholar_has_id_mag,
        join_worked_on_mag,
        COUNT(*) / 1000000.0 AS c_mill,
        COUNT(*) * 100.0 / (SELECT COUNT(*) FROM s2oa_join_on_mag_openalex_id) AS p,
        (SELECT COUNT(*) FROM s2oa_join_on_mag_openalex_id) / 1000000.0 as total_mill
    FROM
        s2oa_join_on_mag_openalex_id
    GROUP BY
        semanticscholar_has_id_mag_or_doi,
        semanticscholar_has_id_doi,
        join_worked_on_doi,
        semanticscholar_has_id_mag,
        join_worked_on_mag
    ORDER BY
        semanticscholar_has_id_mag_or_doi,
        semanticscholar_has_id_doi DESC,
        join_worked_on_doi DESC,
        semanticscholar_has_id_mag DESC,
        join_worked_on_mag DESC
)
SELECT * FROM grouped
 """, '02_stg')

,semanticscholar_has_id_mag_or_doi,semanticscholar_has_id_doi,join_worked_on_doi,semanticscholar_has_id_mag,join_worked_on_mag,c_mill,p,total_mill
0,0,0,0,0,0,0.251800,2.167352,11.617861
1,1,1,1,1,0,6.111962,52.608324,11.617861
2,1,1,1,0,0,4.772088,41.075444,11.617861
3,1,1,0,1,1,0.015118,0.130127,11.617861
4,1,1,0,1,0,0.001548,0.013324,11.617861
5,1,1,0,0,0,0.056051,0.482455,11.617861
6,1,0,0,1,1,0.393961,3.390994,11.617861
7,1,0,0,1,0,0.015333,0.131978,11.617861


Our current results at this stage are:  
 - Line 0: 252K records (2.2% of 11.6M total) don't have either MAG or DOI, so it's impossible to join them
 - Lines 1&2: The previously seen 10.9M records (of which 6.1M had MAG id as well, while 4.8M did not) which is 93.7% of the total were matched on DOI.
 - Line 3: There was only 15K (0.1%) where the record had DOI, but it couldn't join on it, but it could join on MAG.
 - Line 6: However, there is 394K (3.4%) where the record did not have DOI, but it had MAG and it could join on MAG.
 - Line 4, 5, 7: There was altogether 87K records where we had DOI and/or MAG id, but we couldn't join them.

So let's see how these results sum up:

In [14]:
utils.pd_set_options(cols=500)
wr.athena.read_sql_query("""
WITH
base_semanticscholar_s2orcv2_ AS (
    SELECT *, LOWER(id_doi) AS id_doi_lower FROM "02_stg".base_semanticscholar_s2orcv2
),
base_openalex_works_reduced_ AS (
SELECT *, LOWER(id_doi) AS id_doi_lower FROM "02_stg"."base_openalex_works_reduced"
),
flagged_semanticscholar_s2orcv2 AS (
    SELECT
        *,
        CASE WHEN COALESCE(id_mag, '')='' THEN 0 ELSE 1 END AS has_id_mag,
        CASE WHEN COALESCE(id_doi, '')='' THEN 0 ELSE 1 END AS has_id_doi,
        CASE WHEN COALESCE(id_doi, '')='' AND COALESCE(id_mag, '')='' THEN 0 ELSE 1 END AS has_id_mag_or_doi
    FROM
        base_semanticscholar_s2orcv2_
),
s2oa_join_on_doi_id AS (
    SELECT
        flagged_semanticscholar_s2orcv2.title AS semanticscholar_title, 
        flagged_semanticscholar_s2orcv2.id_semanticscholar AS semanticscholar_id_semanticscholar,
        flagged_semanticscholar_s2orcv2.id_mag AS semanticscholar_id_mag,
        flagged_semanticscholar_s2orcv2.id_doi AS semanticscholar_id_doi,                         -- Note: This is case-sensitive
        flagged_semanticscholar_s2orcv2.id_doi_lower AS semanticscholar_id_doi_lower,             -- Note: lowered
        flagged_semanticscholar_s2orcv2.has_id_mag AS semanticscholar_has_id_mag,
        flagged_semanticscholar_s2orcv2.has_id_doi AS semanticscholar_has_id_doi,
        flagged_semanticscholar_s2orcv2.has_id_mag_or_doi AS semanticscholar_has_id_mag_or_doi,
        base_openalex_works_reduced_.title AS openalex_doi_title,
        base_openalex_works_reduced_.id_openalex AS openalex_doi_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_doi_id_doi,                                 -- Note: This is forced lower-case
        base_openalex_works_reduced_.id_doi_lower AS openalex_doi_id_doi_lower,                     -- Note: lowered (just to be sure)
        CASE WHEN base_openalex_works_reduced_.id_doi IS NULL THEN 0 ELSE 1 END AS join_worked_on_doi
    FROM
        flagged_semanticscholar_s2orcv2
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        flagged_semanticscholar_s2orcv2.id_doi_lower = base_openalex_works_reduced_.id_doi_lower
),
s2oa_join_on_mag_openalex_id AS (
    SELECT
        s2oa_join_on_doi_id.semanticscholar_title, 
        s2oa_join_on_doi_id.semanticscholar_id_semanticscholar,
        s2oa_join_on_doi_id.semanticscholar_id_mag,
        s2oa_join_on_doi_id.semanticscholar_id_doi,                         -- Note: This is case-sensitive
        s2oa_join_on_doi_id.semanticscholar_id_doi_lower,                   -- Note: lowered
        s2oa_join_on_doi_id.semanticscholar_has_id_mag,
        s2oa_join_on_doi_id.semanticscholar_has_id_doi,
        s2oa_join_on_doi_id.semanticscholar_has_id_mag_or_doi,

        s2oa_join_on_doi_id.openalex_doi_title,
        s2oa_join_on_doi_id.openalex_doi_id_openalex,
        s2oa_join_on_doi_id.openalex_doi_id_doi,                           -- Note: This is forced lower-case
        s2oa_join_on_doi_id.openalex_doi_id_doi_lower,                     -- Note: lowered (just to be sure)
        s2oa_join_on_doi_id.join_worked_on_doi,
        
        base_openalex_works_reduced_.title AS openalex_mag_title,
        base_openalex_works_reduced_.id_openalex AS openalex_mag_id_openalex,
        base_openalex_works_reduced_.id_doi AS openalex_mag_id_doi,                                 -- Note: This is forced lower-case
        base_openalex_works_reduced_.id_doi_lower AS openalex_mag_id_doi_lower,                     -- Note: lowered (just to be sure)
        CASE WHEN base_openalex_works_reduced_.id_openalex IS NULL THEN 0 ELSE 1 END AS join_worked_on_mag
    FROM
        s2oa_join_on_doi_id
    LEFT JOIN
        base_openalex_works_reduced_
    ON
        s2oa_join_on_doi_id.join_worked_on_doi=0 AND -- only joining those through MAG that we couldn't through DOI
        s2oa_join_on_doi_id.semanticscholar_id_mag = base_openalex_works_reduced_.id_openalex
),
grouped AS (
    SELECT
        join_worked_on_doi,
        join_worked_on_mag,
        COUNT(*) / 1000000.0 AS c_mill,
        COUNT(*) * 100.0 / (SELECT COUNT(*) FROM s2oa_join_on_mag_openalex_id) AS p,
        (SELECT COUNT(*) FROM s2oa_join_on_mag_openalex_id) / 1000000.0 as total_mill
    FROM
        s2oa_join_on_mag_openalex_id
    GROUP BY
        join_worked_on_doi,
        join_worked_on_mag
    ORDER BY
        join_worked_on_doi DESC,
        join_worked_on_mag DESC
)
SELECT * FROM grouped
 """, '02_stg')

,join_worked_on_doi,join_worked_on_mag,c_mill,p,total_mill
0,1,0,10.884050,93.683768,11.617861
1,0,1,0.409079,3.521121,11.617861
2,0,0,0.324732,2.795110,11.617861


At this point we have a match for 97.2% of the records, with only 2.8% or 325K missing, and we can accept this.

Now it's time to put the join into a final usable format.